In [ ]:
data_dir = 'D:\\projects\\ev_detection\\data\\f28e28af-5e8b-44e4-bf01-e85eb1c221fb\\load'

In [ ]:
file1 = '68907590-9b66-477d-9112-36e16be09c24.csv'
file2 = '94466047-4726-4b92-8dcc-2e69aa16debf.csv'

In [ ]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]
print(onlyfiles[0])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
rnd = np.random.randint(0,len(onlyfiles),2)
pd1 = pd.read_csv(join(data_dir,onlyfiles[rnd[0]]),sep=';',parse_dates=[0],date_parser=lambda col: pd.to_datetime(col, utc=True))
pd2 = pd.read_csv(join(data_dir,onlyfiles[rnd[1]]),sep=';',parse_dates=[0],date_parser=lambda col: pd.to_datetime(col, utc=True))

In [ ]:
pd1.dtypes

In [ ]:
pd1.head(4)

In [ ]:
pd1=pd1.set_index('date_tz')
pd2=pd2.set_index('date_tz')
pd1 = pd1[~pd1.index.duplicated()]
pd2 = pd2[~pd2.index.duplicated()]

In [ ]:
m1 = pd1['kWh/h'].mean()
m2 = pd2['kWh/h'].mean()
pd1['kWh/h'] = pd1['kWh/h']-m1
pd2['kWh/h'] = pd2['kWh/h']-m2

In [ ]:
pd1.plot(figsize=[8,8])
pd2.plot(figsize=[8,8])

In [ ]:
pd1_rs = pd1.resample(rule='W',axis=0).mean()
pd2_rs = pd2.resample(rule='W',axis=0).mean()

In [ ]:
pd1_rs.plot(figsize=[8,8])
pd2_rs.plot(figsize=[8,8])

## TSLEARN Sandbox

In [ ]:
from tslearn.utils import to_time_series_dataset,to_time_series

In [ ]:
ts1 = to_time_series(pd1['kWh/h'])
ts2 = to_time_series(pd2['kWh/h'])

In [ ]:
#from tslearn.metrics import dtw

In [ ]:
#ff = dtw(ts1,ts2)

In [ ]:
#ff

## trying to fit seasonal trend

In [ ]:
from PyEMD import EMD,Visualisation
emd1 = EMD()
emd2 = EMD()

In [ ]:
emd1.FIXE = 10
imfs1 = emd1(pd1_rs['kWh/h'])
emd2.FIXE = 10
imfs2 = emd2(pd2_rs['kWh/h'])

In [ ]:
v1=Visualisation()
v1.plot_imfs(imfs=imfs1,include_residue=False)
v2=Visualisation()
v2.plot_imfs(imfs=imfs2,include_residue=False)

In [ ]:
imfs1_pd=pd.DataFrame.from_records(np.transpose(imfs1),index=pd1_rs.index,columns=[f'imf{n}' for n in range(0,imfs1.shape[0])])
imfs2_pd=pd.DataFrame.from_records(np.transpose(imfs2),index=pd2_rs.index,columns=[f'imf{n}' for n in range(0,imfs2.shape[0])])

In [ ]:
pd1_new = pd1_rs.merge(right=imfs1_pd,left_index=True,right_index=True)
pd2_new = pd2_rs.merge(right=imfs2_pd,left_index=True,right_index=True)

In [ ]:
pd1_new.plot(figsize=[8,8])
pd2_new.plot(figsize=[8,8])

In [ ]:
imfs1_pd_hr = imfs1_pd.resample(rule='H',axis=0).interpolate(method='linear')
imfs2_pd_hr = imfs2_pd.resample(rule='H',axis=0).interpolate(method='linear')

In [ ]:
pd1=pd1.merge(right=imfs1_pd_hr,left_index=True,right_index=True)
pd2=pd2.merge(right=imfs2_pd_hr,left_index=True,right_index=True)

In [ ]:
pd1.plot(figsize=[8,8])
pd2.plot(figsize=[8,8])

In [ ]:
pd1['detrend']=pd1['kWh/h']-(pd1[f'imf{imfs1.shape[0]-1}']+pd1[f'imf{imfs1.shape[0]-2}']+pd1[f'imf{imfs1.shape[0]-3}'])
pd2['detrend']=pd2['kWh/h']-(pd2[f'imf{imfs2.shape[0]-1}']+pd2[f'imf{imfs2.shape[0]-2}']+pd2[f'imf{imfs2.shape[0]-3}'])

In [ ]:
pd1.plot(figsize=[12,8])
pd2.plot(figsize=[12,8])

In [ ]:
pd1['detrend_norm']=(pd1['detrend']-pd1['detrend'].min())/(pd1['detrend'].max()-pd1['detrend'].min())
pd2['detrend_norm']=(pd2['detrend']-pd2['detrend'].min())/(pd2['detrend'].max()-pd2['detrend'].min())

In [ ]:
pd1['detrend_norm'].plot(figsize=[12,8])
pd2['detrend_norm'].plot(figsize=[12,8])

## Use Plotly for visualisation

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=pd1.index, y=pd1['detrend_norm'], name="Detrend Norm 1",
                         line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=pd2.index, y=pd2['detrend_norm'], name="Detrend Norm 2",
                         line_color='dimgray'))

fig.update_layout(title_text='Detrend with Rangeslider',
                  xaxis_rangeslider_visible=True)
fig.show()

## Cut in weeks

In [ ]:
fig = plt.figure(figsize=[16,8])
for d in range(0,7):
    test_pd = pd1.loc[pd1.index.dayofweek==d,'detrend_norm']
    mean_cons=[]
    for f in range(0,24):
        mean_cons.append(test_pd.loc[(test_pd.index.hour==f)].mean())
    plt.plot(range(0,24),mean_cons)

fig2 = plt.figure(figsize=[16,8])
for d in range(0,7):
    test_pd = pd2.loc[pd2.index.dayofweek==d,'detrend_norm']
    mean_cons=[]
    for f in range(0,24):
        mean_cons.append(test_pd.loc[(test_pd.index.hour==f)].mean())
    plt.plot(range(0,24),mean_cons)